In [2]:
! pip install progress

  Stored in directory: C:\Users\sahoo\AppData\Local\pip\Cache\wheels\6c\c8\80\32a294e3041f006c661838c05a411c7b7ffc60ff939d14e116
Successfully built progress


In [3]:
import numpy as np
import cv2
import copy
import os
import re
from progress.bar import Bar

In [8]:
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

cv2.startWindowThread()

cap = cv2.VideoCapture(0)
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
out = cv2.VideoWriter( 'output.avi', cv2.VideoWriter_fourcc(*'MJPG'),15., (640,480))
heatmap = cv2.VideoWriter( 'heatmap.avi', cv2.VideoWriter_fourcc(*'MJPG'),15., (640,480))
first_iteration = True
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    out.write(frame.astype('uint8'))

    # resizing for faster detection
    if first_iteration == True:
        first_frame = copy.deepcopy(frame)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        height, width = gray.shape[:2]
        accum_image = np.zeros((height, width), np.uint8)
        first_iteration = False
    
    else:
        thresh = 2
        maxValue = 2
        fgmask = fgbg.apply(frame)
        ret, th1 = cv2.threshold(fgmask, thresh, maxValue, cv2.THRESH_BINARY)
        
        accum_image = cv2.add(accum_image, th1)
        cv2.imwrite('./mask.jpg', accum_image)

        color_image_video = cv2.applyColorMap(accum_image, cv2.COLORMAP_HOT)
        video_frame = cv2.addWeighted(frame, 0.7, color_image_video, 0.7, 0)
    
        # Write the output video 
        heatmap.write(video_frame.astype('uint8'))
        # Display the resulting frame
        cv2.imshow('frame',video_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
                     
            
color_image = cv2.applyColorMap(accum_image, cv2.COLORMAP_HOT)
result_overlay = cv2.addWeighted(first_frame, 0.7, color_image, 0.7, 0)

out.release()
cap.release()
heatmap.release()

cv2.destroyAllWindows()
cv2.waitKey(0)


-1